In [1]:
using Pkg
Pkg.activate(joinpath(pwd(),".."))
using Random, Distributions, PrettyTables, SCS, Plots, LinearAlgebra, Convex
using Base.Threads: @threads, @spawn
Random.seed!(1234)

  Activating project at `~/Desktop/MachineLearning`


TaskLocalRNG()

# (a)

In [2]:
function data(n, k, ρ, σ, β₁, β₂)
    #Set covariance matrix
    Σ = ρ .* ones(k,k)
    Σ[diagind(Σ)] .= 1
    μ = zeros(k)
    mvnormal = MvNormal(μ,Σ)
    X = rand(mvnormal, n)'

    normal = Normal(0, σ)
    U = rand(normal, n)
    x1 = X[:,1]
    x2 = X[:,2]
    Y = 1 .+ β₁ .* x1 + β₂ .* x2 + U

    return( Y = Y, X = X)
end


data (generic function with 1 method)

# (b)

In [3]:
MOI = Convex.MOI
opt = MOI.OptimizerWithAttributes(SCS.Optimizer, "eps_abs"=>1.0e-08, "eps_rel"=>1.0e-08, MOI.Silent() => true)

MathOptInterface.OptimizerWithAttributes(SCS.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("eps_abs") => 1.0e-8, MathOptInterface.RawOptimizerAttribute("eps_rel") => 1.0e-8, MathOptInterface.Silent() => true])

In [4]:
function lasso(Y, X, λ)
    n = length(Y)
    X = [X ones(n,1)]
    k = size(X, 2)

    b = Variable(k)
    Q = X'X / n
    c = X'Y / n                    #c'b = Y'X*b

    L1 = quadform(b, Q)            #b'Q*b
    L2 = dot(c, b)                 #c'b
    L3 = norm(b[1:k-1], 1)                #sum(|b|)
    

    Q = minimize(L1 - 2 * L2 + λ * L3) 
    solve!(Q, opt,verbose=false)

    return b = round.(vec(evaluate(b)),digits=5)
end


lasso (generic function with 1 method)

# (c)

In [5]:
function MC(ρ, σ, β₁, β₂, R, n, k)
    λ = 1.1 * σ * sqrt(2 * log(n*k)/n)
    p1 = 0
    p2 = 0
    p3 = 0
    @threads for i in 1:R
        (Y, X) = data(n, k, ρ, σ, β₁, β₂)
        b = lasso(Y, X, λ)
        p1 += (b[1] > 0)/R
        p2 += (b[2] > 0)/R
        p3 += (norm(b[3:50],1) > 0)/R
    end
    return (p1 = round(p1, digits = 5), p2 = round(p2, digits = 5), p3 = round(p3, digits = 5))
end

MC (generic function with 1 method)

# (e) - (g)

In [6]:
(p11, p12, p13) = MC(0.2, 2, 2, 2, 1000, 300, 50)
(p21, p22, p23) = MC(0.2, 0.1, 0.1, 0.1, 1000, 300, 50)
(p31, p32, p33) = MC(0.2, 2, 2, 0.2, 1000, 300, 50)
(p41, p42, p43) = MC(0.9, 2, 2, 2, 1000, 300, 50)


result = ["Group 1" p11 p12 p13; "Group 2" p21 p22 p23; "Group 3" p31 p32 p33; "Group 4" p41 p42 p43]
header = [" ", "Select Xi,1", "Select Xi,2", "Select Irrelevant Regressors"]
pretty_table(result; header = header)

┌─────────┬─────────────┬─────────────┬──────────────────────────────┐
│         │ Select Xi,1 │ Select Xi,2 │ Select Irrelevant Regressors │
├─────────┼─────────────┼─────────────┼──────────────────────────────┤
│ Group 1 │       0.992 │         1.0 │                        0.758 │
│ Group 2 │       0.997 │       0.997 │                        0.786 │
│ Group 3 │       0.999 │       0.351 │                        0.722 │
│ Group 4 │         1.0 │       0.999 │                        0.992 │
└─────────┴─────────────┴─────────────┴──────────────────────────────┘
